In [87]:
import numpy as np
import os
#import msgpack
#import msgpack_numpy as m
#m.patch()
import time
import datetime
import dateutil.parser
from sklearn.cluster import KMeans
import scipy.spatial.distance
import sys

In [28]:
os.getcwd()

'/Users/q224516/workspace-saga/bigjob/Pilot-InMemory/examples/jupyter/streaming/kmeans'

In [31]:
data = np.loadtxt("../kmeans_100_100_3.np")
print str(data.nbytes) + " bytes"

240000 bytes


# Scikit-Learn

In [6]:
%%timeit
kmeans = KMeans(n_clusters=2, init="random", random_state=0).fit(data)

10 loops, best of 3: 59.6 ms per loop


# Custom Implementation

In [105]:
def closestPoint(points, centers):    
    best_centroid_index=np.argmin(scipy.spatial.distance.cdist(sample, center, 'euclidean'), axis=1)
    #print np.amin(scipy.spatial.distance.cdist(sample, center, 'euclidean'), axis=1)
    return best_centroid_index


def averagePoints(points, best_centroid_index):
    print("Call average points on: " + str(points))
    points_extracted = [eval(i)[1] for i in eval(points)]
    points_np = np.array(points_extracted)
    new_center = np.mean(points_np, axis=0)
    print("New center: " + str(new_center))
    new_center_string = ','.join(['%.5f' % num for num in new_center])
    print("New center string: " + new_center_string)
    return new_center_string


In [106]:
data.shape

(10000, 3)

In [107]:
center=data[1:3]
sample=data[1:10]

In [108]:
print center

[[ 0.40068908  0.24234925  0.16542718]
 [ 0.21317874  0.40440718  0.10781296]]


In [109]:
print sample

[[ 0.40068908  0.24234925  0.16542718]
 [ 0.21317874  0.40440718  0.10781296]
 [ 0.19195375  0.19847674  0.21378974]
 [ 0.01289819  0.27379977  0.37789631]
 [ 0.08774853  0.19826547  0.36920886]
 [ 0.07544883  0.36641404  0.37218645]
 [ 0.34107729  0.22479568  0.2542066 ]
 [-0.10416565  0.21560696  0.29771197]
 [ 0.2304259   0.17877219  0.3403174 ]]


In [113]:
scipy.spatial.distance.cdist(sample, center, 'euclidean')

array([[ 0.        ,  0.25444507],
       [ 0.25444507,  0.        ],
       [ 0.21871025,  0.23257025],
       [ 0.44329904,  0.36071541],
       [ 0.37603476,  0.35574563],
       [ 0.40487367,  0.30051015],
       [ 0.10836734,  0.26466852],
       [ 0.52258273,  0.41522839],
       [ 0.25222644,  0.32444864]])

In [110]:
closestPoint(sample, center)

[ 0.          0.          0.21871025  0.36071541  0.35574563  0.30051015
  0.10836734  0.41522839  0.25222644]


array([0, 1, 0, 1, 1, 1, 0, 1, 0])

In [85]:
sample.shape

(4, 3)

In [92]:
np.argmin(scipy.spatial.distance.cdist(sample, center, 'euclidean'), axis=1)

array([0, 1, 0, 1])

In [74]:
assigned_centers=np.zeros(len(sample))
min_dist=np.empty(len(sample))
min_dist[:]=np.inf
dist = np.sum((sample - center)**2, axis=1)

In [75]:
dist

array([ 0.        ,  0.06474229,  0.04783417,  0.19651404])

In [76]:
min_dist = np.array([0, 3, 3, 4])

In [77]:
np.less(dist, min_dist)

array([False,  True,  True,  True], dtype=bool)

In [78]:
assigned_centers[np.less(dist, min_dist)]=1

In [79]:
assigned_centers

array([ 0.,  1.,  1.,  1.])

In [12]:
dist = np.sum((sample - center) ** 2)

In [13]:
dist

0.309090506309049

# Kafka Integration

In [ ]:


zkKafka='c251-104.wrangler.tacc.utexas.edu:2181'
client = KafkaClient(zookeeper_hosts=zkKafka)
#client = KafkaClient(hosts='c251-142.wrangler.tacc.utexas.edu:9092')
topic = client.topics['latency']
producer = topic.get_sync_producer()
consumer = topic.get_simple_consumer()


# In[5]:

run_timestamp=datetime.datetime.now()
RESULT_FILE= "results/kafka-latency-" + run_timestamp.strftime("%Y%m%d-%H%M%S") + ".csv"
try:
    os.makedirs("results")
except:
    pass
output_file=open(RESULT_FILE, "w")


source_time=1482011522.371088
sink_time=1482011549.907337
TIME_OFFSET=sink_time-source_time

while True:
    message = consumer.consume(block=True)
    #print message.value
    now = time.time()
    #sent_time=datetime.datetime.strptime(message.value, "%Y-%m-%dT%H:%M:%S.%fZ")
    sent_time_string = message.value.split(";")[0]
    sleep_time =float(message.value.split(";")[1])
    sent_time = dateutil.parser.parse(sent_time_string)
    sent_time_ts = time.mktime(sent_time.timetuple())
    lat = now-sent_time_ts - TIME_OFFSET   
    result = "kafka, latency, 0, %.5f, %.5f, %s, %s\n"%(1/sleep_time, lat, message.value.split(";")[0], 
                                                                    datetime.datetime.now().isoformat())
    print(result)
    output_file.write(result)
    output_file.flush()
